# Data & Things

## Lecture : IoT and Data

<hr>
<h5> 7) IoT and sensor data </h5>
<ul>
    <li> There are two hand-in exercises for this topic:</li>
       <ul>
        <li>i. The first one is explained in bullet 2 of slide 16 of the slide deck “IoT & Sensor
        </li>Data.pptx”. You need to do the entire task described in bullet 2 of slide 16.
        <li>ii. The second one is related to PCA and it is explained in the notebook “pdm_task”.
            Note that the data simulation parts are already done for you, so you do not need
            to do those parts. You should start from the cell where tasks are listed and you
        </li> need to do all 5 tasks.
        </ul>
</ul>

<h5> bullet 2 of slide 16</h5>
<ul><li>Now, try youself:<br> Use the diabetes.csv dataset to do the following:</li>
 <ol>
    <li>Select the following 4 attributes(3 features+1 class label):</li>
        <ul><li>Glucose, BloodPreasure, Insulin, Outcome</li></ul>
    <li>Normalize Glucose, BloodPressure and Insulin to [0,1] using MinMax.</li>
    <li>Store the new data(3 normalized features + 1 class label) in another dataset S.</li>
    <li>Modify the MQTT example to do the following:</li>
        <ul>
            <li>The publisher publishes records in 5 continuously, When it reaches the end of S, it continues to send from the beginning again.</li>
            <li>The subscriber continuously receices the data. For each latest record r received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.</li>
            <ul><li>Repeat this for 10000 times. and report the number of correct classifications.</li></ul>
        </ul>
 </ol>
</ul>

<hr>

# Simulate sensor data & apply PCA

In this exercise, you will do the following:
1. Simulate sensor data
2. Apply PCA to simulated data
3. Simulate a new point
4. Find out which region the new point belongs to

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Step 1 Simulate synthetic predictive maintenance dataset
np.random.seed(42)
time_steps = 10000

sensors = {}
for i in range(1, 21):
    sensors[f'Sensor{i}'] = np.linspace(50, 100, time_steps) + np.random.normal(0, 20, time_steps)

# Combine into a DataFrame
data = pd.DataFrame(sensors)

print("Sample Data:")
print(data.head())

Sample Data:
     Sensor1    Sensor2    Sensor3    Sensor4    Sensor5    Sensor6  \
0  59.934283  36.430105  56.965725  10.388561  40.562849  51.964123   
1  47.239714  43.895011  55.671472  28.905288  70.259050  48.722836   
2  62.963772  38.062380  31.279604  38.269433  46.046264  69.045816   
3  80.475599  52.223362  61.606686  53.008380  51.826387  80.671624   
4  45.336935  73.963573  20.218348  70.503249  64.367814  63.756941   

     Sensor7    Sensor8    Sensor9   Sensor10   Sensor11   Sensor12  \
0  51.576755  56.403048  63.280022  69.217891  70.611892  66.017477   
1  61.262950  72.314233  44.082680  42.605706  26.897905  49.021813   
2  56.832040  19.905237  48.678273  38.418383  61.518745  41.079748   
3  24.456718  84.807115  46.507354  68.681245  37.630232  53.375301   
4  46.295596  56.621738  60.301076  -4.610793  43.471946   7.484493   

    Sensor13   Sensor14   Sensor15   Sensor16   Sensor17   Sensor18  \
0  45.626995  72.350652  44.689254  77.397560  78.652455  13.5

In [3]:
# Step 2: Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# Display scaled data
print("Scaled Data Sample:")
print(pd.DataFrame(scaled_data, columns=data.columns).head())

Scaled Data Sample:
    Sensor1   Sensor2   Sensor3   Sensor4   Sensor5   Sensor6   Sensor7  \
0 -0.610580 -1.566006 -0.725010 -2.596654 -1.407431 -0.927666 -0.969437   
1 -1.126525 -1.265030 -0.777771 -1.850742 -0.198347 -1.057996 -0.572396   
2 -0.487452 -1.500195 -1.772111 -1.473524 -1.184173 -0.240819 -0.754021   
3  0.224281 -0.929241 -0.535820 -0.879792 -0.948835  0.226649 -2.081098   
4 -1.203859 -0.052701 -2.223025 -0.175043 -0.438209 -0.453482 -1.185914   

    Sensor8   Sensor9  Sensor10  Sensor11  Sensor12  Sensor13  Sensor14  \
0 -0.746813 -0.479506 -0.232249 -0.186682 -0.367113 -1.184181 -0.110562   
1 -0.106939 -1.258090 -1.320570 -1.955170 -1.054209 -0.697301 -0.772550   
2 -2.214587 -1.071707 -1.491814 -0.554553 -1.375289 -0.412171 -0.701019   
3  0.395468 -1.159753 -0.254195 -1.520984 -0.878208 -1.964016 -1.982653   
4 -0.738018 -0.600323 -3.251518 -1.284652 -2.733470 -0.924990 -0.686938   

   Sensor15  Sensor16  Sensor17  Sensor18  Sensor19  Sensor20  
0 -1.220616  0

Do the following tasks:
1. Apply PCA with upto 5 components
2. Visualize how much variance of the dataset is defined by each component
3. Visualize the first 2 principal components and color them by timestep
4. Simulate a new position using code below
5. Plot the new point in the original 2D PCA graph

In [ ]:
# Step 4: Simulate a new observation and predict its position
new_observation = np.array([[75] * 20])
scaled_new_observation = scaler.transform(new_observation)
new_pca = pca.transform(scaled_new_observation)

print("New observation PCA position:", new_pca)

<h1>Task from slides</h1>

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

In [5]:
df = pd.read_csv("diabetes.csv")
display(df.head())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


<h3>Task 1 Select the following 4 attributes (3 features+1 class label): Glucose, BLoodPreasure, Insulin, Outcome</h3>

In [8]:
df_selected = df[['Glucose', 'BloodPressure', 'Insulin', 'Outcome']].copy()

<h3>Task 2 Normalize Glucose, BloodPressure and Insulin to [0,1] using MinMax <h3>

In [9]:
scaler = MinMaxScaler()
df_selected[['Glucose', 'BloodPressure', 'Insulin']] = scaler.fit_transform(df_selected[['Glucose', 'BloodPressure', 'Insulin']])

<h3>Task 3 Store the new data(3 normalized features + 1 class label) in another dataset S.</h3>

<hr>
<h5> 7) IoT and sensor data </h5>
<ul>
    <li> There are two hand-in exercises for this topic:</li>
       <ul>
        <li>i. The first one is explained in bullet 2 of slide 16 of the slide deck “IoT & Sensor
        </li>Data.pptx”. You need to do the entire task described in bullet 2 of slide 16.
        <li>ii. The second one is related to PCA and it is explained in the notebook “pdm_task”.
            Note that the data simulation parts are already done for you, so you do not need
            to do those parts. You should start from the cell where tasks are listed and you
        </li> need to do all 5 tasks.
        </ul>
</ul>

<h5> bullet 2 of slide 16</h5>
<ul><li>Now, try youself:<br> Use the diabetes.csv dataset to do the following:</li>
 <ol>
    <li>Select the following 4 attributes(3 features+1 class label):</li>
        <ul><li>Glucose, BloodPreasure, Insulin, Outcome</li></ul>
    <li>Normalize Glucose, BloodPressure and Insulin to [0,1] using MinMax.</li>
    <li>Store the new data(3 normalized features + 1 class label) in another dataset S.</li>
    <li>Modify the MQTT example to do the following:</li>
        <ul>
            <li>The publisher publishes records in 5 continuously, When it reaches the end of S, it continues to send from the beginning again.</li>
            <li>The subscriber continuously receices the data. For each latest record r received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.</li>
            <ul><li>Repeat this for 10000 times. and report the number of correct classifications.</li></ul>
        </ul>
 </ol>
</ul>

<hr>